# TensorFlow Workbook - Example
This example uses the testing data put together by J. Czerniak and H.Zarzycki
See "Application of rough sets in the presumptive diagnosis of urinary system diseases, Artificial Intelligence and Security in Computing Systems,: ACS'2002 9th International Conference Proceedings, Kluwer Academic Publishers,2003, pp. 41-51.  
See also "Accute Inflammations" at OpenML http://www.openml.org/d/1455

> # Select Data File
Assumes Linux environment with Jupyter Notebook and TensorFlow installed  
Easiest to install Jupyter with Anaconda.  See http://jupyter.org/install.html  
TensorFlow is pretty painless too.  See https://www.tensorflow.org/get_started/os_setup  
Place the csv data file in the current directory:  
Assumes:   
-commas separated fields  
-column headers, also separated by columns  
-no empty or NaN values  
-all text fields converted to numbers  
-target (dependent variable) is in final column  

In [1]:
import tensorflow
import numpy
import pandas
import ipywidgets as widgets
from IPython.display import display
csv_file_list = !ls | egrep '.csv'
d_csv_choice = widgets.Dropdown(
    options = csv_file_list,
    values = csv_file_list, 
    description = 'Available .csv\'s',
)
display(d_csv_choice)

> # Inspect The Data
Make sure your data looks right  
Pandas library will assign an index on the right; all good

In [6]:
df = pandas.read_csv(d_csv_choice.value, header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0
5,8,10,10,8,7,10,9,7,1,1
6,1,1,1,1,2,10,3,1,1,0
7,2,1,2,1,2,1,3,1,1,0
8,2,1,1,1,2,1,1,1,5,0
9,4,2,1,1,2,1,2,1,1,0


> # Select 75/35 Split for Training/Test Data
This splits the labeled data into Training and Testing batches

In [7]:
training_df = df.sample(frac = .75)
training_df
training_indices_list = list(training_df.index)
the_range = len(df)
total_indices_list = list(range(the_range))
testing_indices_list = list(set(total_indices_list) - set(training_indices_list))
print(training_indices_list)
print(testing_indices_list)
testing_df = df.drop(df.index[training_indices_list])
testing_rows = len(testing_df)
training_rows = len(training_df)
training_df.to_csv('training_data.csv', index = False, header = False)
testing_df.to_csv('testing_data.csv', index = False, header = False)
the_columns = len(df.columns) - 1
training_length = len(training_df.index)
testing_length = len(testing_df.index)
print(the_columns)
##testing_df = df.drop(training_df)
##y = len(testing_df)
##print(y)

[171, 229, 318, 262, 595, 652, 462, 137, 158, 659, 405, 183, 446, 352, 223, 543, 151, 197, 374, 667, 4, 395, 26, 20, 569, 73, 563, 256, 259, 67, 398, 636, 553, 37, 319, 305, 15, 337, 533, 218, 602, 348, 541, 243, 577, 438, 642, 460, 176, 192, 372, 591, 572, 452, 289, 678, 21, 182, 428, 247, 130, 153, 172, 396, 206, 194, 329, 60, 542, 664, 617, 278, 671, 547, 681, 189, 209, 18, 110, 566, 646, 551, 508, 178, 619, 586, 516, 445, 230, 580, 668, 127, 350, 96, 529, 578, 480, 159, 166, 688, 415, 203, 24, 618, 108, 324, 333, 544, 557, 526, 43, 116, 175, 605, 234, 103, 196, 656, 409, 401, 471, 132, 565, 477, 435, 447, 302, 378, 149, 81, 52, 346, 698, 266, 496, 249, 369, 325, 362, 163, 601, 660, 135, 361, 493, 70, 581, 2, 363, 525, 71, 84, 501, 380, 430, 680, 662, 607, 439, 392, 332, 40, 136, 562, 1, 497, 419, 504, 22, 111, 478, 157, 188, 425, 515, 44, 429, 254, 600, 635, 294, 675, 468, 588, 371, 268, 343, 235, 74, 265, 587, 90, 509, 404, 354, 432, 427, 448, 211, 95, 472, 483, 638, 210, 390, 590